# Dirichlet boundary conditions

This section shows how to solve the adv diff
$$
-\nabla(\varepsilon \nabla u)+\beta\cdot\nabla u = f \text{ in } \Omega
$$
$$
u=0 \text{ on } \partial\Omega
$$

### Finite element spaces with Dirichlet conditions

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

In [2]:
maxh = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
mesh.GetBoundaries()

('bottom', 'right', 'top', 'left')

In [78]:
fes = H1(mesh, order=1, dirichlet="bottom|top|left|right")
u = fes.TrialFunction()
v = fes.TestFunction()

In [79]:
eps = 0.1
beta = (70,0)
cbeta = CoefficientFunction(beta)
f = 1

### Standard Formulation
$$ \varepsilon\int\nabla u\cdot\nabla v + \int \beta\cdot\nabla u v = \int fv$$

In [80]:
a = BilinearForm(fes)
a += SymbolicBFI(eps*grad(u)*grad(v)+cbeta*grad(u)*v)
a.Assemble()

l = LinearForm(fes)
l += SymbolicLFI(f*v)
l.Assemble()

In [81]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * l.vec 
Draw(gfu)

### Strongly Consistent Stabilization
approximate $h_K$ with $\max_{K} h_K$

##### SUPG $\rho=0$
add $$+\sum_K \delta_K\int_K (-\varepsilon\nabla u + \beta\cdot\nabla u - f)(\beta\cdot\nabla v)\frac{h_k}{|\beta|}$$

In [82]:
delta = 0.3
nbeta = sqrt(beta[0]**2+beta[1]**2)
lapu = u.Operator("hesse")[0] + u.Operator("hesse")[3]
lapv = v.Operator("hesse")[0] + v.Operator("hesse")[3]


a = BilinearForm(fes)
a += SymbolicBFI(eps*grad(u)*grad(v)+cbeta*grad(u)*v)
a += SymbolicBFI(delta*(-eps*lapu*(maxh/nbeta)*cbeta*grad(v)))
a += SymbolicBFI(delta*(cbeta*grad(u)*(maxh/nbeta)*cbeta*grad(v)))
a.Assemble()

l = LinearForm(fes)
l += SymbolicLFI(f*v)
l += SymbolicLFI(delta*( f * (maxh/nbeta)*(cbeta*grad(v)) ))
l.Assemble()

In [83]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * l.vec 
Draw(gfu)